In [1]:
import cv2
import numpy as np
MASK_COLOR = (1.0,1.0,1.0) # In BGR format

In [31]:
#-- Read image -----------------------------------------------------------------------
img = cv2.imread('food_solo.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imwrite('gray.png', gray)

True

In [20]:
#-- 에지 검출 -------------------------------------------------------------------
'''
cv2.Canny(이미지, 하위 임계값, 상위 임계값)
하위 임곗값과 상위 임곗값으로 픽셀이 갖는 최솟값과 최댓값을 설정해 검출 진행
픽셀이 상위 임곗값보다 큰 기울기를 가지면 픽셀을 가장자리로 
간주하고, 하위 임곗값보다 낮은 경우 가장자리로 고려 x
'''
edges = cv2.Canny(gray,40, 150)
#cv2.imshow('Canny_edge', edges)                                   # Display
#cv2.waitKey()

cv2.imwrite('Canny_edge.png', edges)

#이미지 팽창, erode와 반대, 굵어지는 효과
edges = cv2.dilate(edges, None)
#cv2.imshow('Canny_dilate', edges)                                   # Display
#cv2.waitKey()

cv2.imwrite('Canny_dilate.png', edges)

#이미지 침식, 흐릿한 경계부분을 배경으로 만드는 작업, 가늘어지는 효과
edges = cv2.erode(edges, None)
#cv2.imshow('Canny_erode', edges)                                   # Display
#cv2.waitKey()

cv2.imwrite('Canny_erode.png', edges)

True

In [29]:
#-- 에지의 윤곽 검출 및 영역 정렬---------------------------------------------
contour_info = []
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
for c in contours:
    contour_info.append((
        c,
        cv2.contourArea(c),
    ))
contour_info = sorted(contour_info, key=lambda c: c[1], reverse=True)
max_contour = contour_info[0]

img_draw_contour=cv2.drawContours(gray, max_contour[0], -1, (0,255,0), 3)
cv2.imwrite('img_draw_contour.png', img_draw_contour)

True

In [23]:
#-- 빈 마스크를 작성하고 가장 큰 윤곽선에 해당하는 채워진 다각형 생성  ----
# 마스크 = 검정색, 다각형 = 하얀색
mask = np.zeros(edges.shape)
cv2.fillConvexPoly(mask, max_contour[0], (255))
#cv2.imshow('fill_mask', mask)                                   # Display
#cv2.waitKey()

cv2.imwrite('fill_mask.png', mask)

True

In [24]:
#-- 마스크 스무딩 후 블러 처리 --------------------------------------------------------
mask = cv2.dilate(mask, None, iterations=10) #MASK_DILATE_ITER
#cv2.imshow('mask_dilate', mask)                                   # Display
#cv2.waitKey()

cv2.imwrite('mask_dilate.png', mask)

mask = cv2.erode(mask, None, iterations=10)  #MASK_ERODE_ITER
#cv2.imshow('mask_erode', mask)                                   # Display
#cv2.waitKey()

cv2.imwrite('mask_erode.png', mask)

mask = cv2.GaussianBlur(mask, (21, 21), 0)
#cv2.imshow('mask_Gaussian', mask)                                   # Display
#cv2.waitKey()

cv2.imwrite('mask_Gaussian.png', mask)

mask_stack = np.dstack([mask]*3)    # Create 3-channel alpha mask

In [25]:
#-- 마스크된 img를 MASK_COLOR 배경에 혼합 --------------------------------------
mask_stack  = mask_stack.astype('float32') / 255.0          # Use float matrices, 
img         = img.astype('float32') / 255.0                 #  for easy blending

#오브젝트 추출
masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR) # Blend
masked = (masked * 255).astype('uint8')                     # Convert back to 8-bit 
#cv2.imshow('img_obj', masked)                                   # Display
#cv2.waitKey()

#배경 추출
back_masked = ((1-mask_stack) * img) + ((mask_stack) * MASK_COLOR)
back_masked = (back_masked * 255).astype('uint8')
#cv2.imshow('img_bg', back_masked)                                   # Display
#cv2.waitKey()

#이미지 저장
cv2.imwrite('only_coffee.png', masked)           # Save
cv2.imwrite('background.png', back_masked)           

True

In [26]:
#배경 블러처리
img_blur = cv2.blur(back_masked,(13,13))
#cv2.imshow('img_blur', img_blur)                                   # Display
#cv2.waitKey()

cv2.imwrite('img_blur.png', img_blur)  

True

In [27]:
#이미지 합성
#img_blur + masked
bit_and = cv2.bitwise_and(img_blur, masked)
#cv2.imshow('img_and', bit_and)                                   # Display
#cv2.waitKey()
cv2.imwrite('mixed.png', bit_and)

True